### 实际上按照指引, 你应该实现的是这个版本的代码

In [ ]:
# 主函数逐步补充为这个样子
import numpy
import scipy.special
# 这里是主函数的定义, 定义为参数方式, 使用起来更方便
def nn(inputnode, hiddenode, outputnode, learnrate):
    # 定义参数, 网络分为三层, 这里定义每层的节点数--------------
    nn.inputnode=inputnode # 输入层网络节点数量
    nn.hiddenode=hiddenode # 隐藏层节点数量
    nn.outputnode=outputnode # 输出层节点数量
    # 正态分布初始化
    nn.ih = numpy.random.normal(0.0, pow(nn.hiddenode, -0.5),(nn.hiddenode, nn.inputnode))
    nn.ho = numpy.random.normal(0.0, pow(nn.outputnode, -0.5),(nn.outputnode, nn.hiddenode))
    # 启动函数 activation function 就是节点运算的函数
    nn.active_f=lambda x: scipy.special.expit(x)
    nn.learnrate=learnrate # 这个比率是学习的速率
    # 查询函数, 参数是一个list(数组)
    # 未来调用: query([1.0,0.5,-1.5])
    def query(inpl): # inpl是输入向量
        # ----------这里是我偷偷插入的一行, 没什么可讲的, 就是把一维数组(向量), 整理成为二维矩阵
        inp=numpy.array(inpl, ndmin=2).T
        # 矩阵乘法如此简单
        hli=numpy.dot(nn.ih, inp)   # hidden level input
        hlo = nn.active_f(hli) # hidden level output 隐藏层输出等于用s函数处理一下输入
        # 输出层, 一毛一样
        oli=numpy.dot(nn.ho, hlo)
        olo = nn.active_f(oli)
        return olo
    nn.query=query
    # 此句保证可以这样调用: nn.query([1.0,0.5,-1.5])
    def train(inpl,tarl): # inpl是输入向量, tarl是校验向量
        # -----和query一毛一样的5行代码------
        inp=numpy.array(inpl, ndmin=2).T
        hli=numpy.dot(nn.ih, inp)   
        hlo = nn.active_f(hli) 
        oli=numpy.dot(nn.ho, hlo)   
        olo = nn.active_f(oli) 
        # ------从这里开始是不一样的代码了------------
        # 增加了目标数组
        tar=numpy.array(tarl, ndmin=2).T
        # 因为是训练函数, 所以, 我们是有预期结果的, 这种有预期结果的训练就叫: 有监督训练
        # nn.learnrate=0.5 # 之前设置的这个比率真的用在了这里, 这个是学习的速率

        # 运算结果就需要做反向传播调整了
        ## 拿到误差
        output_error=tar -olo # 最终结果误差
        # python做矩阵减法, 非常直观

        hidden_error=numpy.dot(nn.ho.T, output_error) 
        # hidden层的误差, 这里又是一次矩阵乘法

        ## 修正权重
        nn.ho += nn.learnrate*numpy.dot( output_error*(olo*(1-olo)), numpy.transpose(hlo))
        # 还记得吗? nn.learnrate 就是一开始设置的学习比率, 这里才用到
        # 主运算符 a+=b 和其他语言一致, 代表 a=a+b, 意思就是隐藏到输出的那个矩阵做一次加法, 对自身进行修正
        # 隐藏层到输出层的权重矩阵 += 学习速率* 矩阵乘法, 这个矩阵乘法的解释如下: 
        # olo*(1-olo)这个是一个S函数, 然后作用于输出误差矩阵, 还记得吗? 启动函数active_f用的也是他
        # hlo要进行转置才能和output_error结果误差做矩阵乘法(点乘)

        # 如果理解了上面那一句, 那么下面是输入到隐藏层的权重调整, 几乎一毛一样
        nn.ih += nn.learnrate*numpy.dot( hidden_error*(hlo*(1-hlo)), numpy.transpose(inp))
    nn.train=train

In [ ]:
#导入数据文件
dfile=open("./mnist_train_100.csv",'r')
# 读取数据, readlines函数按照[行]读入所有数据
dlist=dfile.readlines()
# 关闭文件
dfile.close()

# 引入绘图包
import matplotlib.pyplot as plt
# 读第一条数据, 并且格式化成为数组(list)
a=dlist[0].split(',')# 这里dlist[0]代表第一条, 可以换换其他数字试试
print(a[0]) # 看一下这个数组的第一个元素, 这个元素是这个数组实际的数字值


In [ ]:
# 初始化nn函数
nn(28*28, 100, 10, 0.3)
# 训练所有数据
for re in dlist:
    a=re.split(',') # 把字符串用逗号拆成数组
    # 这里是把颜色的数字压缩到0-1之间, 颜色数值一般是0-255, 如果不缩数字持续乘法会导致过大, 大到超过计算机能处理的极限
    ia=(numpy.asfarray(a[1:])/255.0*0.99+0.01)
    # 制造训练的target数据
    targetv=numpy.zeros(nn.outputnode) +0.01 # 整体都初始化为0.01
    # 因为第一个数字是这一行数据的数字, 所以用这个数字做target是非常合理的
    targetv[int(a[0])]=0.99 # 只有目标的值是0.99, 其他值应该都是0.01
    # 开始训练
    nn.train(ia, targetv)
# 好了, 可以测试运行一下, 看看是否有啥拼写错误